In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from math import sqrt
import xgboost as xgb
import time
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from tabulate import tabulate 



train_data = pd.read_csv('/kaggle/input/finaldf/final_df.csv')


train_data.head()

,id,num_backspaces,duration_backspaces,num_cut/copy/paste,discarded_text,D/I Ratio,Proportion_of_Deletions,no_distant_revision,no_immediate_revision,major_edits_count,...,Total_Pause_Time_within_word,Mean_Pause_Time_within_word,SD_Pause_Time_within_word,Total_Pause_Time_Before_Sentences,Mean_Pause_Time_Before_Sentences,SD_Pause_Time_Before_Sentences,Total_Pause_Time_in_sentences,Mean_Pause_Time_in_sentences,SD_Pause_Time_in_sentences,score
0,001519c8,417.0,34130.0,417,489,0.243284,0.164512,2517,40.0,93,...,685007,1918.787115,7273.768138,3643,173.476190,416.665168,1181819,56277.095238,43455.913672,3.5
1,0022f953,260.0,23550.0,261,266,0.138287,0.097316,2354,100.0,45,...,1087710,2781.867008,16954.357992,69412,4627.466667,8278.455203,581673,38778.200000,32758.883810,3.5
2,0042269b,439.0,32905.0,439,935,0.266003,0.221984,4037,99.0,173,...,843679,1528.403986,9793.226830,87089,4147.095238,15826.690943,1259632,59982.476190,58697.726407,6.0
3,0059420b,152.0,18411.0,152,163,0.134202,0.092302,1473,83.0,19,...,745392,3067.456790,13400.325753,1466851,112834.692308,365509.369147,1032626,79432.769231,48355.056728,2.0
4,0075873a,517.0,40199.0,517,517,0.266220,0.164806,2490,41.0,124,...,1039641,3208.768519,11181.242502,142133,6179.695652,13762.720184,1073221,46661.782609,73950.739406,4.0


In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_data.drop(['score', 'id'], axis=1),
    train_data['score'],
    test_size=0.2,
    random_state=42
)

X_train.head()
y_train.head()

1610    4.5
1525    5.0
2366    2.5
859     3.0
497     2.0
Name: score, dtype: float64

In [4]:
# Try out some values to get the best hyper parameters
def train_and_evaluate(X_train, y_train, X_valid, y_valid, max_depth, learning_rate, n_estimators):

    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', seed=42, max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    return rmse

# Initialize variables to keep track of the best hyperparameters and lowest RMSE
best_hyperparams = {'max_depth': 0, 'learning_rate': 0, 'n_estimators': 0}
lowest_rmse = float('inf')

for depth in range(5, 17):
    for rate in np.arange(0.01, 0.41, 0.03):
        for n_estimators in range(50, 151, 3):
            rmse = train_and_evaluate(X_train, y_train, X_valid, y_valid, max_depth=depth, learning_rate=rate, n_estimators=n_estimators)
            
            # Update best hyperparameters if the current RMSE is lower
            if rmse < lowest_rmse:
                lowest_rmse = rmse
                best_hyperparams['max_depth'] = depth
                best_hyperparams['learning_rate'] = rate
                best_hyperparams['n_estimators'] = n_estimators
            
            print(f"Max Depth: {depth}, Learning Rate: {rate:.2f}, n_estimators: {n_estimators}, RMSE: {rmse:.4f}, Lowest RMSE: {lowest_rmse:.4f}")     

Max Depth: 5, Learning Rate: 0.01, n_estimators: 50, RMSE: 0.8443, Lowest RMSE: 0.8443
Max Depth: 5, Learning Rate: 0.01, n_estimators: 53, RMSE: 0.8361, Lowest RMSE: 0.8361
Max Depth: 5, Learning Rate: 0.01, n_estimators: 56, RMSE: 0.8280, Lowest RMSE: 0.8280
Max Depth: 5, Learning Rate: 0.01, n_estimators: 59, RMSE: 0.8206, Lowest RMSE: 0.8206
Max Depth: 5, Learning Rate: 0.01, n_estimators: 62, RMSE: 0.8134, Lowest RMSE: 0.8134
Max Depth: 5, Learning Rate: 0.01, n_estimators: 65, RMSE: 0.8067, Lowest RMSE: 0.8067
Max Depth: 5, Learning Rate: 0.01, n_estimators: 68, RMSE: 0.8003, Lowest RMSE: 0.8003
Max Depth: 5, Learning Rate: 0.01, n_estimators: 71, RMSE: 0.7942, Lowest RMSE: 0.7942
Max Depth: 5, Learning Rate: 0.01, n_estimators: 74, RMSE: 0.7883, Lowest RMSE: 0.7883
Max Depth: 5, Learning Rate: 0.01, n_estimators: 77, RMSE: 0.7826, Lowest RMSE: 0.7826
Max Depth: 5, Learning Rate: 0.01, n_estimators: 80, RMSE: 0.7774, Lowest RMSE: 0.7774
Max Depth: 5, Learning Rate: 0.01, n_estima

KeyboardInterrupt: 